In [1]:
import numpy as np
import pandas as pd
from enum import Enum
from abc import ABC, abstractmethod

In [2]:
class PersonState(Enum):
    waiting = 0
    assigned = 1
    riding = 2
    arrived = 3


class CarState(Enum):
    idle = 0
    to_start = 1
    to_end = 2


class Person:
    
    def __init__(self, id_, s, t, time_init):
        self.id = id_
        self.s = s
        self.t = t
        self.state = PersonState.waiting
        self.time_init = time_init
    
    def assign(self, time_assign: int):
        self.state = PersonState.assigned
        self.time_assign = time_assign
    
    def start_ride(self, time_start: int):
        self.state = PersonState.riding
        self.time_start = time_start
    
    def finish_ride(self, time_finish: int):
        self.state = PersonState.arrived
        self.time_finish = time_finish
        
    def where_to(self):
        if self.state == PersonState.waiting:
            return self.s
        elif self.state == PersonState.riding:
            return self.t
        return None
    
    def stats(self):
        if self.state == PersonState.arrived:
            return pd.DataFrame({
                'assigning': [self.time_assign - self.time_init],
                'waiting': [self.time_start - self.time_assign],
                'riding': [self.time_finish - self.time_start]
            }, index = [self.id])
    
    def __repr__(self):
        return f'Person from {self.s} to {self.t}, appeared at {self.time_init}'


class PersonFactory:
    
    def __init__(self):
        self.max_id = -1
    
    def generate_person(self, nodes, time):
        self.max_id += 1
        return Person(self.max_id, *np.random.choice(nodes, 2, replace=False), time)

    
class Car:
    
    def __init__(self, id_: int, start_node: int, time_init: int, limit: int):
        self.id = id_
        self.at = start_node
        self.state = CarState.idle
        self.limit = limit
        self.busy_until = time_init - 1
        self.passengers = [None] * limit
        self.current_person = None
    
    def ride_to_start(self, person: Person, time: int, distance: float):
        self.current_person = person
        self.state = CarState.to_start
        person.assign(time)
        self.at = person.s
        self.busy_until = time + int(distance)
    
    def ride_to_end(self, person: Person, time: int, distance: float):
        self.current_person = person
        self.state = CarState.to_end
        self.at = person.t
        self.busy_until = time + int(distance)
    
    def pick_up(self, time: int):
        self.passengers[self.passengers.index(None)] = self.current_person
        self.current_person.start_ride(time)
        self.current_person = None
    
    def drop_off(self, time: int):
        try:
            self.passengers[self.passengers.index(self.current_person)] = None
        except ValueError:
            print(self.passengers, self.current_person)
            exit(0)
        self.current_person.finish_ride(time)
        self.current_person = None
    
    def is_full(self) -> bool:
        return all(p is not None for p in self.passengers)
    
    def free_check(self, time: int) -> bool:
        if time >= self.busy_until:
            if self.state == CarState.to_start:
                self.pick_up(time)
            elif self.state == CarState.to_end:
                self.drop_off(time)
            self.state = CarState.idle
        return self.state == CarState.idle
            

class CarFactory:
    
    def __init__(self):
        self.max_id = -1
    
    def generate_car(self, nodes, time):
        self.max_id += 1
        return Car(
            self.max_id,
            np.random.choice(nodes),
            time,
            4
        )


class Policy(ABC):
    
    def __init__(self, cars: list, people: list):
        self.cars = cars
        self.people = people
    
    def score(self, car_id: int, time: int) -> np.array:
        non_target = [p.state != PersonState.waiting and p not in self.cars[car_id].passengers for p in self.people]
        scores = self._score(car_id, time)
        scores[non_target] = -np.inf
        return scores
    
    @abstractmethod
    def _score(self, car_id: int, time: int) -> np.array:
        pass


class DummyPolicy(Policy):
    
    def _score(self, car_id: int, time: int) -> list:
        return np.random.rand(len(self.people))


class Environment:
    
    def __init__(self, cars: list, people: list, policy: Policy, distances: pd.DataFrame):
        self.cars = cars
        self.people = people
        self.policy = policy
        self.distances = distances
    
    def step(self, time: int):
        for car in self.cars:
            if car.free_check(time):
                scores = np.array(self.policy.score(car.id, time))
                if scores.max() == -np.inf:
                    return
                if car.is_full():
                    person_id = np.argmax(scores[tuple(p.id for p in car.passengers)])
                else:
                    person_id = np.argmax(scores)
                person = self.people[person_id]
                if person.state == PersonState.waiting:
                    car.ride_to_start(person, time, distances.loc[[(car.at, person.s)]]['dst'])
                elif person.state == PersonState.riding:
                    car.ride_to_end(person, time, distances.loc[[(car.at, person.t)]]['dst'])
    
    def stats(self):
        return pd.concat([p.stats() for p in self.people if p.stats() is not None])

In [3]:
time = 0
car_proba  = 0.005
pers_proba = 0.05
nodes = [1, 2, 3]
distances = pd.DataFrame({'dst': [1, 2, 3, 4, 3, 2, 0, 0, 0]}, 
                         index=[(1, 2), (2, 3), (3, 2), (2, 1), (1, 3), (3, 1), (1, 1), (2, 2), (3, 3)])

car_f = CarFactory()
per_f = PersonFactory()

cars = [car_f.generate_car(nodes, time)]
people = [per_f.generate_person(nodes, time), per_f.generate_person(nodes, time)]

policy = DummyPolicy(cars, people)
environment = Environment(cars, people, policy, distances)

while time <= 10 ** 4:
    if np.random.rand() < car_proba:
        cars.append(car_f.generate_car(nodes, time))
    if np.random.rand() < pers_proba:
        people.append(per_f.generate_person(nodes, time))
    environment.step(time)
    time += 1
    if time % 1000 == 0:
        print(time, 'time units passed')

1000 time units passed
2000 time units passed
3000 time units passed
4000 time units passed
5000 time units passed
6000 time units passed
7000 time units passed
8000 time units passed
9000 time units passed
10000 time units passed


In [4]:
environment.stats().mean()

assigning    0.008048
waiting      3.967807
riding       4.911469
dtype: float64